# churn prediction using logisitic regression

In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score

In [2]:
#loading data
data=pd.DataFrame()
link="C:/Users/lavanya/Desktop/churn_prediction.csv"
path=pd.read_csv(link)
path

,customer_id,vintage,age,gender,dependents,occupation,city,customer_nw_category,branch_code,days_since_last_transaction,...,previous_month_end_balance,average_monthly_balance_prevQ,average_monthly_balance_prevQ2,current_month_credit,previous_month_credit,current_month_debit,previous_month_debit,current_month_balance,previous_month_balance,churn
0,1,3135,66,Male,0.0,self_employed,187.0,2,755,224.0,...,1458.71,1458.71,1449.07,0.20,0.20,0.20,0.20,1458.71,1458.71,0
1,2,310,35,Male,0.0,self_employed,NaN,2,3214,60.0,...,8704.66,7799.26,12419.41,0.56,0.56,5486.27,100.56,6496.78,8787.61,0
2,4,2356,31,Male,0.0,salaried,146.0,2,41,NaN,...,5815.29,4910.17,2815.94,0.61,0.61,6046.73,259.23,5006.28,5070.14,0
3,5,478,90,NaN,NaN,self_employed,1020.0,2,582,147.0,...,2291.91,2084.54,1006.54,0.47,0.47,0.47,2143.33,2291.91,1669.79,1
4,6,2531,42,Male,2.0,self_employed,1494.0,3,388,58.0,...,1401.72,1643.31,1871.12,0.33,714.61,588.62,1538.06,1157.15,1677.16,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28377,30297,1845,10,Female,0.0,student,1020.0,2,1207,70.0,...,1076.43,2282.19,2787.70,0.30,0.30,0.30,0.30,1076.43,1076.43,0
28378,30298,4919,34,Female,0.0,self_employed,1046.0,2,223,14.0,...,4069.21,3668.83,3865.55,1.71,2.29,901.00,1014.07,3738.54,3690.32,0
28379,30299,297,47,Male,0.0,salaried,1096.0,2,588,0.0,...,61017.55,53444.81,21925.81,4666.84,3883.06,168.23,71.80,61078.50,57564.24,1
28380,30300,2585,50,Male,3.0,self_employed,1219.0,3,274,NaN,...,1625.55,1683.20,1857.42,0.20,0.20,0.20,0.20,1625.55,1625.55,0


In [3]:
path["churn"].value_counts()

0    23122
1     5260
Name: churn, dtype: int64

## Handling missing values

In [4]:
path.isnull().sum()

customer_id                          0
vintage                              0
age                                  0
gender                             525
dependents                        2463
occupation                          80
city                               803
customer_nw_category                 0
branch_code                          0
days_since_last_transaction       3223
current_balance                      0
previous_month_end_balance           0
average_monthly_balance_prevQ        0
average_monthly_balance_prevQ2       0
current_month_credit                 0
previous_month_credit                0
current_month_debit                  0
previous_month_debit                 0
current_month_balance                0
previous_month_balance               0
churn                                0
dtype: int64

In [5]:
path['gender'].value_counts()

Male      16548
Female    11309
Name: gender, dtype: int64

In [6]:
path.replace("Male",0,inplace=True)
path.replace("Female",1,inplace=True)
path['gender'] = path['gender'].fillna(-1)

In [7]:
path['gender'].fillna(-1)

0        0.0
1        0.0
2        0.0
3       -1.0
4        0.0
        ... 
28377    1.0
28378    1.0
28379    0.0
28380    0.0
28381    0.0
Name: gender, Length: 28382, dtype: float64

In [8]:
path['dependents'].value_counts()

0.0     21435
2.0      2150
1.0      1395
3.0       701
4.0       179
5.0        41
6.0         8
7.0         3
36.0        1
52.0        1
25.0        1
9.0         1
50.0        1
32.0        1
8.0         1
Name: dependents, dtype: int64

In [9]:
path['occupation'].value_counts()

self_employed    17476
salaried          6704
student           2058
retired           2024
company             40
Name: occupation, dtype: int64

In [10]:
path['dependents'] = path['dependents'].fillna(0)
path['occupation'] = path['occupation'].fillna('self_employed')

In [11]:
path['city'].value_counts()

1020.0    3479
1096.0    2016
409.0     1334
146.0     1291
834.0     1138
          ... 
629.0        1
527.0        1
1212.0       1
530.0        1
70.0         1
Name: city, Length: 1604, dtype: int64

In [12]:
path['city'] = path['city'].fillna(1020)

In [13]:
path['days_since_last_transaction'].value_counts()

0.0      1672
3.0       831
14.0      654
4.0       632
6.0       596
         ... 
170.0       1
218.0       1
334.0       1
288.0       1
321.0       1
Name: days_since_last_transaction, Length: 360, dtype: int64

In [14]:
path['days_since_last_transaction'] = path['days_since_last_transaction'].fillna(0)

In [15]:
path

,customer_id,vintage,age,gender,dependents,occupation,city,customer_nw_category,branch_code,days_since_last_transaction,...,previous_month_end_balance,average_monthly_balance_prevQ,average_monthly_balance_prevQ2,current_month_credit,previous_month_credit,current_month_debit,previous_month_debit,current_month_balance,previous_month_balance,churn
0,1,3135,66,0.0,0.0,self_employed,187.0,2,755,224.0,...,1458.71,1458.71,1449.07,0.20,0.20,0.20,0.20,1458.71,1458.71,0
1,2,310,35,0.0,0.0,self_employed,1020.0,2,3214,60.0,...,8704.66,7799.26,12419.41,0.56,0.56,5486.27,100.56,6496.78,8787.61,0
2,4,2356,31,0.0,0.0,salaried,146.0,2,41,0.0,...,5815.29,4910.17,2815.94,0.61,0.61,6046.73,259.23,5006.28,5070.14,0
3,5,478,90,-1.0,0.0,self_employed,1020.0,2,582,147.0,...,2291.91,2084.54,1006.54,0.47,0.47,0.47,2143.33,2291.91,1669.79,1
4,6,2531,42,0.0,2.0,self_employed,1494.0,3,388,58.0,...,1401.72,1643.31,1871.12,0.33,714.61,588.62,1538.06,1157.15,1677.16,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28377,30297,1845,10,1.0,0.0,student,1020.0,2,1207,70.0,...,1076.43,2282.19,2787.70,0.30,0.30,0.30,0.30,1076.43,1076.43,0
28378,30298,4919,34,1.0,0.0,self_employed,1046.0,2,223,14.0,...,4069.21,3668.83,3865.55,1.71,2.29,901.00,1014.07,3738.54,3690.32,0
28379,30299,297,47,0.0,0.0,salaried,1096.0,2,588,0.0,...,61017.55,53444.81,21925.81,4666.84,3883.06,168.23,71.80,61078.50,57564.24,1
28380,30300,2585,50,0.0,3.0,self_employed,1219.0,3,274,0.0,...,1625.55,1683.20,1857.42,0.20,0.20,0.20,0.20,1625.55,1625.55,0


In [16]:
path.isnull().sum()

customer_id                       0
vintage                           0
age                               0
gender                            0
dependents                        0
occupation                        0
city                              0
customer_nw_category              0
branch_code                       0
days_since_last_transaction       0
current_balance                   0
previous_month_end_balance        0
average_monthly_balance_prevQ     0
average_monthly_balance_prevQ2    0
current_month_credit              0
previous_month_credit             0
current_month_debit               0
previous_month_debit              0
current_month_balance             0
previous_month_balance            0
churn                             0
dtype: int64

In [17]:
path['occupation'].unique()

array(['self_employed', 'salaried', 'retired', 'student', 'company'],
      dtype=object)

In [18]:
path.replace("self_employed",1,inplace=True)
path.replace("salaried",2,inplace=True)
path.replace("retired",3,inplace=True)
path.replace("student",4,inplace=True)
path.replace("company",5,inplace=True)
path

,customer_id,vintage,age,gender,dependents,occupation,city,customer_nw_category,branch_code,days_since_last_transaction,...,previous_month_end_balance,average_monthly_balance_prevQ,average_monthly_balance_prevQ2,current_month_credit,previous_month_credit,current_month_debit,previous_month_debit,current_month_balance,previous_month_balance,churn
0,1,3135,66,0.0,0.0,1,187.0,2,755,224.0,...,1458.71,1458.71,1449.07,0.20,0.20,0.20,0.20,1458.71,1458.71,0
1,2,310,35,0.0,0.0,1,1020.0,2,3214,60.0,...,8704.66,7799.26,12419.41,0.56,0.56,5486.27,100.56,6496.78,8787.61,0
2,4,2356,31,0.0,0.0,2,146.0,2,41,0.0,...,5815.29,4910.17,2815.94,0.61,0.61,6046.73,259.23,5006.28,5070.14,0
3,5,478,90,-1.0,0.0,1,1020.0,2,582,147.0,...,2291.91,2084.54,1006.54,0.47,0.47,0.47,2143.33,2291.91,1669.79,1
4,6,2531,42,0.0,2.0,1,1494.0,3,388,58.0,...,1401.72,1643.31,1871.12,0.33,714.61,588.62,1538.06,1157.15,1677.16,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28377,30297,1845,10,1.0,0.0,4,1020.0,2,1207,70.0,...,1076.43,2282.19,2787.70,0.30,0.30,0.30,0.30,1076.43,1076.43,0
28378,30298,4919,34,1.0,0.0,1,1046.0,2,223,14.0,...,4069.21,3668.83,3865.55,1.71,2.29,901.00,1014.07,3738.54,3690.32,0
28379,30299,297,47,0.0,0.0,2,1096.0,2,588,0.0,...,61017.55,53444.81,21925.81,4666.84,3883.06,168.23,71.80,61078.50,57564.24,1
28380,30300,2585,50,0.0,3.0,1,1219.0,3,274,0.0,...,1625.55,1683.20,1857.42,0.20,0.20,0.20,0.20,1625.55,1625.55,0


## Correlation between columns

In [19]:
path.corr()

,customer_id,vintage,age,gender,dependents,occupation,city,customer_nw_category,branch_code,days_since_last_transaction,...,previous_month_end_balance,average_monthly_balance_prevQ,average_monthly_balance_prevQ2,current_month_credit,previous_month_credit,current_month_debit,previous_month_debit,current_month_balance,previous_month_balance,churn
customer_id,1.000000,-0.006093,-0.000442,0.004525,-0.009110,-0.003013,0.000669,0.009618,-0.000286,0.009579,...,0.005819,0.004485,-0.002532,0.002494,-0.006414,0.002603,-0.008760,0.005140,0.004553,-0.002723
vintage,-0.006093,1.000000,0.227390,-0.020868,0.068588,-0.032801,0.041720,-0.167366,-0.380714,-0.078002,...,0.004518,0.003433,0.002244,0.002848,0.028034,0.005406,0.023949,0.001638,0.003448,-0.064637
age,-0.000442,0.227390,1.000000,-0.165955,0.001261,-0.227543,0.014564,-0.076532,-0.058990,-0.010413,...,0.058342,0.061708,0.059607,0.023840,0.029961,0.027702,0.033296,0.057662,0.060297,-0.020012
gender,0.004525,-0.020868,-0.165955,1.000000,-0.153341,-0.063690,-0.005627,-0.024127,-0.060009,0.047798,...,-0.037675,-0.037465,-0.035158,-0.014667,-0.028765,-0.019984,-0.033521,-0.038965,-0.036521,-0.020769
dependents,-0.009110,0.068588,0.001261,-0.153341,1.000000,-0.085979,0.003287,0.007917,-0.001704,-0.061647,...,-0.002181,-0.001786,-0.001967,0.003542,0.025848,0.008225,0.030746,-0.002769,-0.001672,0.029335
occupation,-0.003013,-0.032801,-0.227543,-0.063690,-0.085979,1.000000,-0.002379,-0.005865,-0.053314,0.011168,...,-0.020887,-0.021995,-0.020657,-0.001604,-0.021595,-0.007553,-0.030603,-0.020825,-0.021731,-0.041849
city,0.000669,0.041720,0.014564,-0.005627,0.003287,-0.002379,1.000000,0.009323,-0.055779,-0.001705,...,-0.004634,-0.006799,-0.008213,0.003802,0.007335,0.001231,0.005448,-0.006355,-0.006318,-0.000883
customer_nw_category,0.009618,-0.167366,-0.076532,-0.024127,0.007917,-0.005865,0.009323,1.000000,0.235059,0.041783,...,-0.059854,-0.059535,-0.047010,-0.025254,-0.072374,-0.035917,-0.071721,-0.058648,-0.059113,0.006551
branch_code,-0.000286,-0.380714,-0.058990,-0.060009,-0.001704,-0.053314,-0.055779,0.235059,1.000000,0.044414,...,0.000214,0.001955,0.001310,-0.013988,-0.023849,-0.016944,-0.017584,0.001031,0.002080,0.035469
days_since_last_transaction,0.009579,-0.078002,-0.010413,0.047798,-0.061647,0.011168,-0.001705,0.041783,0.044414,1.000000,...,-0.010703,-0.009077,-0.004412,-0.027754,-0.058708,-0.031839,-0.055935,-0.013774,-0.008291,-0.029452


In [20]:
x=path[["current_month_debit","previous_month_debit","current_month_balance","previous_month_balance","days_since_last_transaction","vintage"]]
y=path["churn"]
model=LogisticRegression()
model.fit(x,y)
x_train,x_test,y_train,y_test=train_test_split(x,y)
y_pred=model.predict(x_test)
y_pred

C:\Users\lavanya\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [21]:
model.score(x_test,y_test)

0.8215896279594137

In [22]:
model.score(x_train,y_train)

0.8139622286949169

In [23]:
x_test

,current_month_debit,previous_month_debit,current_month_balance,previous_month_balance,days_since_last_transaction,vintage
5746,0.63,0.63,6666.62,6666.62,0.0,2407
6028,928.87,1786.01,6362.25,7427.27,116.0,682
21979,0.27,5840.65,3900.86,3354.88,52.0,4456
20150,75.26,8214.80,5062.55,4462.73,6.0,1225
20453,0.43,0.43,111556.72,111556.72,0.0,1546
...,...,...,...,...,...,...
19155,1428.74,2857.31,794.41,2520.14,83.0,807
21257,701.83,495.94,3461.65,9445.92,47.0,3229
25505,40.14,323.03,10112.46,10839.45,77.0,1558
20051,0.70,0.70,5866.33,5866.33,180.0,2543


In [24]:
model.predict([[5486.27,100.56,6496.78,8787.61,60,310]])

array([0], dtype=int64)

In [25]:
model.predict([[5000,0,5000,5000,10,342]])

array([0], dtype=int64)

In [26]:
model.predict([[0,30000,0,30000,89,654]])

array([1], dtype=int64)

In [27]:
model.score(x_test,y_pred)

1.0

In [44]:
d=path.query('churn==0 and age>=25 and age<=30')
d

,customer_id,vintage,age,gender,dependents,occupation,city,customer_nw_category,branch_code,days_since_last_transaction,...,previous_month_end_balance,average_monthly_balance_prevQ,average_monthly_balance_prevQ2,current_month_credit,previous_month_credit,current_month_debit,previous_month_debit,current_month_balance,previous_month_balance,churn
75,79,765,26,1.0,0.0,1,698.0,3,231,19.0,...,2013.86,2145.23,2001.11,571.57,0.14,0.14,0.14,2407.95,2013.86,0
91,95,2964,30,0.0,0.0,2,58.0,2,803,75.0,...,3148.70,3380.35,6022.71,0.47,0.47,0.47,0.47,3143.23,3148.70,0
92,96,1644,29,1.0,0.0,2,409.0,2,26,219.0,...,8450.77,7659.27,6097.42,0.17,0.17,366.39,48.89,8201.65,7761.98,0
140,148,1525,30,0.0,0.0,1,142.0,2,1950,164.0,...,1638.88,1638.54,2879.81,0.54,0.54,0.54,0.54,1642.60,1638.88,0
141,149,1039,30,1.0,0.0,1,409.0,2,679,110.0,...,2348.97,3645.54,7706.60,0.20,0.20,0.20,3285.91,2348.97,3049.50,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28338,30256,1314,30,0.0,0.0,1,1096.0,3,557,7.0,...,4062.16,3361.29,1139.63,643.56,643.56,766.91,13.97,3622.86,3875.03,0
28353,30272,2215,25,1.0,0.0,4,1589.0,2,1712,37.0,...,1591.98,1524.49,1464.64,0.51,118.93,0.51,0.51,1591.98,1507.94,0
28362,30282,1011,26,0.0,0.0,4,146.0,3,1745,219.0,...,7007.92,7036.29,7278.40,0.64,0.64,890.66,89.71,6786.47,7086.96,0
28363,30283,3409,26,1.0,0.0,4,334.0,2,82,0.0,...,7422.94,7422.94,7375.91,0.01,0.01,0.01,0.01,7422.94,7422.94,0


In [45]:
d=path.query('churn==1 and age>=25 and age<=30')
d

,customer_id,vintage,age,gender,dependents,occupation,city,customer_nw_category,branch_code,days_since_last_transaction,...,previous_month_end_balance,average_monthly_balance_prevQ,average_monthly_balance_prevQ2,current_month_credit,previous_month_credit,current_month_debit,previous_month_debit,current_month_balance,previous_month_balance,churn
21,24,2083,29,1.0,0.0,1,1020.0,2,1106,0.0,...,5149.74,5569.58,4826.60,0.47,0.47,357.61,786.19,5029.05,5744.21,1
49,52,1776,30,1.0,0.0,1,409.0,2,2072,7.0,...,16242.22,14497.05,16252.35,3260.71,0.67,14617.16,1743.53,11112.23,16259.74,1
70,74,761,27,0.0,1.0,1,1525.0,2,194,46.0,...,3572.74,2623.92,394.73,0.37,5714.66,3191.36,7339.07,1546.76,4973.61,1
83,87,1106,26,0.0,0.0,1,1154.0,3,1730,126.0,...,483.52,2032.88,6547.79,0.30,0.30,0.30,57.44,483.52,560.64,1
95,99,2629,26,0.0,0.0,1,318.0,3,35,30.0,...,3187.91,1440.99,1260.99,0.34,1085.49,2877.35,410.67,2173.64,1634.38,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28079,29975,2494,29,0.0,0.0,2,1540.0,2,347,17.0,...,237.16,1997.51,3815.47,113.47,430.39,30.57,3739.94,271.53,2308.62,1
28095,29993,502,25,1.0,0.0,1,146.0,3,77,59.0,...,2108.42,2358.90,2131.30,0.16,519.61,846.34,437.76,1533.27,2223.79,1
28240,30151,2623,30,1.0,0.0,1,1020.0,2,117,48.0,...,1979.12,2279.95,4026.98,0.33,286.04,0.33,1191.76,1624.84,2089.42,1
28313,30229,3927,26,0.0,0.0,1,15.0,2,461,53.0,...,364.78,1709.33,744.02,0.10,214.39,28.67,3714.39,347.04,1102.11,1


In [46]:
d.count()

customer_id                       494
vintage                           494
age                               494
gender                            494
dependents                        494
occupation                        494
city                              494
customer_nw_category              494
branch_code                       494
days_since_last_transaction       494
current_balance                   494
previous_month_end_balance        494
average_monthly_balance_prevQ     494
average_monthly_balance_prevQ2    494
current_month_credit              494
previous_month_credit             494
current_month_debit               494
previous_month_debit              494
current_month_balance             494
previous_month_balance            494
churn                             494
dtype: int64

In [47]:
d=path.query('churn==0 and age>=30 and age<=35')
d

,customer_id,vintage,age,gender,dependents,occupation,city,customer_nw_category,branch_code,days_since_last_transaction,...,previous_month_end_balance,average_monthly_balance_prevQ,average_monthly_balance_prevQ2,current_month_credit,previous_month_credit,current_month_debit,previous_month_debit,current_month_balance,previous_month_balance,churn
1,2,310,35,0.0,0.0,1,1020.0,2,3214,60.0,...,8704.66,7799.26,12419.41,0.56,0.56,5486.27,100.56,6496.78,8787.61,0
2,4,2356,31,0.0,0.0,2,146.0,2,41,0.0,...,5815.29,4910.17,2815.94,0.61,0.61,6046.73,259.23,5006.28,5070.14,0
8,10,2132,31,0.0,0.0,2,1096.0,2,4110,19.0,...,1355.86,1702.44,1126.13,714.51,121.09,714.51,359.44,1183.45,1857.42,0
11,13,7108,32,0.0,0.0,2,1096.0,1,89,20.0,...,8372.64,9149.45,21007.52,7564.53,4550.24,9907.91,5161.96,8532.97,8072.33,0
23,26,1897,34,0.0,0.0,1,931.0,2,671,4.0,...,1401.93,1657.70,2357.17,2072.09,0.66,1191.06,1786.37,1185.47,2098.38,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28350,30269,400,32,1.0,0.0,1,1637.0,2,965,0.0,...,1542.07,1541.78,1532.31,0.56,0.56,25.99,0.56,1541.19,1542.07,0
28365,30285,3853,35,0.0,0.0,1,223.0,2,187,0.0,...,2535.64,2616.38,3212.75,0.56,0.56,214.84,0.56,2454.36,2535.64,0
28367,30287,2401,34,1.0,0.0,1,409.0,3,1566,49.0,...,10034.12,9782.20,9404.43,0.31,571.74,0.31,0.31,10034.12,9849.79,0
28371,30291,3999,32,1.0,0.0,2,1096.0,2,132,27.0,...,1319.88,1526.78,1310.70,643.34,0.49,0.49,0.49,1940.57,1319.88,0


In [48]:
d=path.query('churn==0 and age>=35 and age<=40')
d

,customer_id,vintage,age,gender,dependents,occupation,city,customer_nw_category,branch_code,days_since_last_transaction,...,previous_month_end_balance,average_monthly_balance_prevQ,average_monthly_balance_prevQ2,current_month_credit,previous_month_credit,current_month_debit,previous_month_debit,current_month_balance,previous_month_balance,churn
1,2,310,35,0.0,0.0,1,1020.0,2,3214,60.0,...,8704.66,7799.26,12419.41,0.56,0.56,5486.27,100.56,6496.78,8787.61,0
9,11,3379,40,0.0,3.0,1,1020.0,2,38,0.0,...,4244.38,2168.53,1864.33,6912.29,4206.13,1887.30,1348.85,1823.16,2416.69,0
26,29,2620,36,0.0,2.0,1,1563.0,1,58,20.0,...,23106.03,26221.15,5594.83,7.83,722.68,853.09,1557.17,21741.48,25504.19,0
80,84,762,40,1.0,0.0,1,745.0,3,2042,138.0,...,10074.20,10165.31,10167.53,0.40,0.40,0.40,0.40,10024.94,10161.76,0
88,92,733,35,0.0,0.0,2,575.0,3,1002,0.0,...,2226.87,2205.30,5458.80,0.29,0.29,0.29,0.29,2757.94,2177.22,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28351,30270,2658,40,0.0,0.0,2,531.0,2,483,115.0,...,7568.05,7024.44,728.56,0.14,0.14,387.13,143.00,7550.60,7621.12,0
28357,30276,1264,40,0.0,1.0,1,1020.0,2,47,12.0,...,2392.05,4899.22,4388.74,20298.14,171.97,14316.02,747.71,9069.99,2659.88,0
28358,30277,4087,36,0.0,0.0,1,1020.0,2,398,9.0,...,3283.84,2628.89,2880.20,2037.67,2099.24,257.85,79.10,3920.31,2881.12,0
28359,30278,3377,39,0.0,0.0,2,409.0,3,331,195.0,...,1257.99,2323.91,1227.16,0.44,0.44,422.73,4997.51,1232.52,3098.41,0


In [50]:
d=path.query('churn==0 and age>=25 and age<=30 and gender==0')
d

,customer_id,vintage,age,gender,dependents,occupation,city,customer_nw_category,branch_code,days_since_last_transaction,...,previous_month_end_balance,average_monthly_balance_prevQ,average_monthly_balance_prevQ2,current_month_credit,previous_month_credit,current_month_debit,previous_month_debit,current_month_balance,previous_month_balance,churn
91,95,2964,30,0.0,0.0,2,58.0,2,803,75.0,...,3148.70,3380.35,6022.71,0.47,0.47,0.47,0.47,3143.23,3148.70,0
140,148,1525,30,0.0,0.0,1,142.0,2,1950,164.0,...,1638.88,1638.54,2879.81,0.54,0.54,0.54,0.54,1642.60,1638.88,0
146,154,2909,28,0.0,0.0,1,32.0,3,820,46.0,...,4689.26,4597.46,4302.62,0.09,194.11,0.09,0.09,4689.26,4607.89,0
147,155,1395,28,0.0,0.0,1,146.0,2,1759,14.0,...,1660.06,2233.48,4640.59,185.77,2257.20,726.49,3128.63,2974.86,1844.77,0
165,177,2491,26,0.0,0.0,2,146.0,1,76,6.0,...,3289.97,3352.04,4172.94,2.36,0.64,647.28,304.37,2725.90,3497.39,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28271,30185,519,25,0.0,0.0,2,1096.0,3,292,158.0,...,1872.02,1936.28,1441.81,0.13,0.13,401.39,500.13,2182.21,2024.10,0
28320,30236,874,25,0.0,0.0,2,285.0,2,770,68.0,...,2943.26,2950.04,3370.26,0.17,0.17,571.60,785.89,2849.66,3007.77,0
28338,30256,1314,30,0.0,0.0,1,1096.0,3,557,7.0,...,4062.16,3361.29,1139.63,643.56,643.56,766.91,13.97,3622.86,3875.03,0
28362,30282,1011,26,0.0,0.0,4,146.0,3,1745,219.0,...,7007.92,7036.29,7278.40,0.64,0.64,890.66,89.71,6786.47,7086.96,0


In [51]:
d=path.query('churn==0 and age>=25 and age<=30 and gender==1')
d

,customer_id,vintage,age,gender,dependents,occupation,city,customer_nw_category,branch_code,days_since_last_transaction,...,previous_month_end_balance,average_monthly_balance_prevQ,average_monthly_balance_prevQ2,current_month_credit,previous_month_credit,current_month_debit,previous_month_debit,current_month_balance,previous_month_balance,churn
75,79,765,26,1.0,0.0,1,698.0,3,231,19.0,...,2013.86,2145.23,2001.11,571.57,0.14,0.14,0.14,2407.95,2013.86,0
92,96,1644,29,1.0,0.0,2,409.0,2,26,219.0,...,8450.77,7659.27,6097.42,0.17,0.17,366.39,48.89,8201.65,7761.98,0
141,149,1039,30,1.0,0.0,1,409.0,2,679,110.0,...,2348.97,3645.54,7706.60,0.20,0.20,0.20,3285.91,2348.97,3049.50,0
161,172,592,29,1.0,0.0,1,146.0,2,2870,359.0,...,23967.13,23527.00,21781.23,0.01,0.01,119.30,0.01,23872.53,23967.13,0
179,194,1389,28,1.0,0.0,1,318.0,2,1379,135.0,...,17510.91,20106.09,18523.31,0.51,0.51,6786.23,7719.09,14074.95,21041.14,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28265,30178,2087,30,1.0,0.0,2,1034.0,3,41,0.0,...,8765.20,8349.94,14153.67,0.19,0.19,0.19,0.19,6597.56,8765.20,0
28273,30187,264,25,1.0,0.0,1,623.0,2,632,23.0,...,11664.16,7281.63,6031.50,286.19,12143.33,0.47,7857.61,5462.19,9233.28,0
28314,30230,819,26,1.0,0.0,1,1456.0,2,1860,0.0,...,4024.53,4006.83,3965.82,0.39,0.39,0.39,0.39,4024.53,4005.73,0
28353,30272,2215,25,1.0,0.0,4,1589.0,2,1712,37.0,...,1591.98,1524.49,1464.64,0.51,118.93,0.51,0.51,1591.98,1507.94,0


In [53]:
d=path.query('churn==0 and age>=30 and age<=35 and gender==0')
d

,customer_id,vintage,age,gender,dependents,occupation,city,customer_nw_category,branch_code,days_since_last_transaction,...,previous_month_end_balance,average_monthly_balance_prevQ,average_monthly_balance_prevQ2,current_month_credit,previous_month_credit,current_month_debit,previous_month_debit,current_month_balance,previous_month_balance,churn
1,2,310,35,0.0,0.0,1,1020.0,2,3214,60.0,...,8704.66,7799.26,12419.41,0.56,0.56,5486.27,100.56,6496.78,8787.61,0
2,4,2356,31,0.0,0.0,2,146.0,2,41,0.0,...,5815.29,4910.17,2815.94,0.61,0.61,6046.73,259.23,5006.28,5070.14,0
8,10,2132,31,0.0,0.0,2,1096.0,2,4110,19.0,...,1355.86,1702.44,1126.13,714.51,121.09,714.51,359.44,1183.45,1857.42,0
11,13,7108,32,0.0,0.0,2,1096.0,1,89,20.0,...,8372.64,9149.45,21007.52,7564.53,4550.24,9907.91,5161.96,8532.97,8072.33,0
23,26,1897,34,0.0,0.0,1,931.0,2,671,4.0,...,1401.93,1657.70,2357.17,2072.09,0.66,1191.06,1786.37,1185.47,2098.38,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28286,30200,1225,35,0.0,0.0,1,608.0,2,405,25.0,...,18738.08,18690.14,7290.00,571.46,571.46,490.74,490.74,19405.89,18668.72,0
28296,30210,2639,31,0.0,3.0,2,1232.0,2,148,0.0,...,1800.81,1800.81,1789.41,0.37,0.37,0.37,0.37,1800.81,1800.81,0
28298,30212,1175,35,0.0,0.0,1,1110.0,3,2293,126.0,...,1651.89,1696.77,846.13,0.21,0.21,0.21,0.21,1651.89,1714.56,0
28338,30256,1314,30,0.0,0.0,1,1096.0,3,557,7.0,...,4062.16,3361.29,1139.63,643.56,643.56,766.91,13.97,3622.86,3875.03,0


In [54]:
d=path.query('churn==0 and age>=35 and age<=40 and gender==1')
d

,customer_id,vintage,age,gender,dependents,occupation,city,customer_nw_category,branch_code,days_since_last_transaction,...,previous_month_end_balance,average_monthly_balance_prevQ,average_monthly_balance_prevQ2,current_month_credit,previous_month_credit,current_month_debit,previous_month_debit,current_month_balance,previous_month_balance,churn
80,84,762,40,1.0,0.0,1,745.0,3,2042,138.0,...,10074.20,10165.31,10167.53,0.40,0.40,0.40,0.40,10024.94,10161.76,0
106,111,302,39,1.0,0.0,1,465.0,3,2468,259.0,...,8125.81,8863.14,16568.76,0.01,0.01,0.01,0.01,8125.81,8125.81,0
136,144,1476,35,1.0,0.0,1,572.0,3,2397,102.0,...,1072.56,1994.21,1781.40,0.21,0.21,0.21,1428.79,1072.56,2408.96,0
150,158,822,40,1.0,0.0,2,243.0,2,2396,3.0,...,258.02,3978.78,3041.39,2143.13,5143.13,0.27,14285.99,827.07,4705.02,0
155,165,2468,36,1.0,0.0,1,1525.0,3,665,48.0,...,1861.02,1664.98,1513.34,0.53,929.10,350.69,0.53,1701.32,1986.71,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28276,30190,2859,40,1.0,0.0,1,531.0,2,588,352.0,...,2462.71,2462.71,2670.80,0.66,0.66,0.66,0.66,2462.71,2462.71,0
28277,30191,861,36,1.0,0.0,1,1096.0,2,249,85.0,...,3586.22,4438.32,3373.51,0.37,0.37,0.37,0.37,3586.22,3466.40,0
28287,30201,894,39,1.0,0.0,1,1111.0,3,2649,7.0,...,3114.00,2579.55,1399.19,10.00,912.90,1.15,172.44,3089.35,2687.28,0
28299,30213,2613,38,1.0,0.0,1,834.0,2,105,0.0,...,2291.24,2339.20,2533.82,0.57,0.57,0.57,0.57,2271.53,2337.32,0


In [55]:
d=path.query('churn==0 and age>=40 and age<=45 and gender==0')
d

,customer_id,vintage,age,gender,dependents,occupation,city,customer_nw_category,branch_code,days_since_last_transaction,...,previous_month_end_balance,average_monthly_balance_prevQ,average_monthly_balance_prevQ2,current_month_credit,previous_month_credit,current_month_debit,previous_month_debit,current_month_balance,previous_month_balance,churn
9,11,3379,40,0.0,3.0,1,1020.0,2,38,0.0,...,4244.38,2168.53,1864.33,6912.29,4206.13,1887.30,1348.85,1823.16,2416.69,0
46,49,4526,41,0.0,0.0,1,52.0,2,654,11.0,...,1536.52,1577.99,1795.98,1428.73,7828.73,3928.73,7143.01,751.91,2606.82,0
51,55,2701,43,0.0,0.0,2,480.0,3,249,95.0,...,6617.24,7897.49,8096.35,0.51,0.51,2143.37,2714.80,5434.97,8096.50,0
61,65,4870,45,0.0,0.0,2,1322.0,2,176,11.0,...,11695.56,11506.59,8441.80,342.91,285.77,0.06,0.06,11935.11,11566.53,0
76,80,1672,41,0.0,0.0,1,409.0,3,921,12.0,...,747.55,2028.69,436.32,298.60,1515.79,1191.11,1925.96,5028.71,389.58,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28335,30253,1586,42,0.0,0.0,1,621.0,3,2294,54.0,...,1600.61,1489.97,2154.09,0.23,450.73,857.37,571.66,825.77,1687.57,0
28351,30270,2658,40,0.0,0.0,2,531.0,2,483,115.0,...,7568.05,7024.44,728.56,0.14,0.14,387.13,143.00,7550.60,7621.12,0
28355,30274,5116,45,0.0,2.0,1,1540.0,1,1259,21.0,...,1415.38,4840.63,730.14,3645.04,4487.00,18150.47,2563.26,13131.23,842.84,0
28357,30276,1264,40,0.0,1.0,1,1020.0,2,47,12.0,...,2392.05,4899.22,4388.74,20298.14,171.97,14316.02,747.71,9069.99,2659.88,0


In [56]:
d=path.query('churn==0 and age>=45 and age<=50 and gender==1')
d

,customer_id,vintage,age,gender,dependents,occupation,city,customer_nw_category,branch_code,days_since_last_transaction,...,previous_month_end_balance,average_monthly_balance_prevQ,average_monthly_balance_prevQ2,current_month_credit,previous_month_credit,current_month_debit,previous_month_debit,current_month_balance,previous_month_balance,churn
18,21,5821,47,1.0,1.0,1,146.0,1,490,69.0,...,3965.26,4356.62,3623.20,0.56,0.56,71.99,8600.56,3605.40,4022.04,0
50,54,1957,45,1.0,0.0,1,1096.0,2,438,0.0,...,9885.21,9885.21,9821.38,0.17,0.17,0.17,0.17,9885.21,9885.21,0
62,66,704,45,1.0,0.0,1,15.0,2,890,18.0,...,5104.17,4925.66,2604.71,947.69,947.69,0.67,572.10,5688.03,4772.01,0
139,147,5905,49,1.0,0.0,1,1096.0,2,503,40.0,...,7398.11,9804.13,4836.96,0.17,412.88,857.31,1285.89,6786.50,9262.38,0
160,171,518,49,1.0,0.0,1,1043.0,2,2037,3.0,...,3303.75,3348.25,3311.84,1057.36,0.21,357.36,0.21,3437.24,3303.75,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28289,30203,3348,45,1.0,1.0,1,11.0,1,121,43.0,...,1648.85,1504.76,1727.94,0.31,431.83,249.37,249.37,1459.91,1496.61,0
28292,30206,456,50,1.0,0.0,1,892.0,3,2890,0.0,...,1558.93,1493.25,1549.06,0.60,0.60,286.31,0.60,1361.89,1558.93,0
28326,30243,1167,49,1.0,0.0,2,409.0,2,1565,14.0,...,3673.77,3325.84,2206.79,557.16,559.59,0.49,0.49,3942.51,3202.21,0
28333,30251,1136,50,1.0,0.0,1,1598.0,3,220,0.0,...,4870.82,3498.47,63.49,0.13,0.13,0.13,14.41,4873.09,4882.15,0


In [57]:
d=path.query('churn==0 and age>=50 and age<=60 and gender==0')
d

,customer_id,vintage,age,gender,dependents,occupation,city,customer_nw_category,branch_code,days_since_last_transaction,...,previous_month_end_balance,average_monthly_balance_prevQ,average_monthly_balance_prevQ2,current_month_credit,previous_month_credit,current_month_debit,previous_month_debit,current_month_balance,previous_month_balance,churn
13,15,5703,50,0.0,0.0,2,409.0,1,88,303.0,...,3526.01,2831.89,8075.81,0.61,0.61,1748.08,2529.75,2985.16,1536.98,0
27,30,1391,56,0.0,0.0,1,836.0,3,2421,9.0,...,6971.29,7040.32,7298.14,143.26,0.40,3.26,0.40,7019.56,6971.29,0
28,31,4175,55,0.0,0.0,1,118.0,1,325,9.0,...,7666.31,7003.05,6322.76,63.90,18898.87,165.46,14515.19,6499.62,8589.60,0
32,35,1124,54,0.0,0.0,1,1366.0,2,797,0.0,...,3294.42,3294.42,3273.68,0.36,0.36,0.36,0.36,3294.42,3294.42,0
35,38,2684,52,0.0,7.0,1,549.0,2,264,125.0,...,8387.76,3587.43,3397.86,0.19,0.19,1090.94,16700.25,3656.30,5707.49,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28290,30204,734,58,0.0,0.0,1,1563.0,3,2616,58.0,...,2163.39,1757.15,796.99,0.06,75.77,0.06,0.06,2163.39,2160.95,0
28330,30247,4890,51,0.0,2.0,1,1096.0,2,89,47.0,...,10940.09,9582.84,8262.24,0.66,714.94,4872.14,1712.91,7681.86,10807.45,0
28334,30252,1537,52,0.0,2.0,1,1629.0,2,1448,20.0,...,1653.70,2351.72,393.14,3083.13,0.41,0.41,0.41,3886.01,1653.70,0
28349,30268,4653,58,0.0,0.0,1,996.0,3,656,0.0,...,5119.61,5119.61,5125.47,0.50,0.50,0.50,0.50,5119.61,5119.61,0


In [58]:
d=path.query('churn==0 and age>=50 and age<=60 and gender==1')
d

,customer_id,vintage,age,gender,dependents,occupation,city,customer_nw_category,branch_code,days_since_last_transaction,...,previous_month_end_balance,average_monthly_balance_prevQ,average_monthly_balance_prevQ2,current_month_credit,previous_month_credit,current_month_debit,previous_month_debit,current_month_balance,previous_month_balance,churn
15,17,1934,51,1.0,0.0,1,1232.0,3,149,0.0,...,1994.84,1829.43,2415.57,0.16,0.16,128.66,360.70,1960.91,1840.23,0
17,20,6111,52,1.0,0.0,1,1096.0,2,32,3.0,...,3804.06,3946.99,3931.91,3916.91,0.27,0.27,16.70,4158.66,3810.95,0
43,46,2938,55,1.0,0.0,1,146.0,1,509,100.0,...,3315.42,3568.09,3893.81,0.27,0.27,715.99,2328.84,3308.91,3918.18,0
54,58,4843,51,1.0,0.0,1,1020.0,2,15,20.0,...,610.75,2002.71,1029.63,487.46,487.46,120.31,3334.60,871.43,1369.55,0
98,102,1856,56,1.0,0.0,1,1534.0,2,63,39.0,...,22627.30,14741.84,7979.38,0.39,14643.24,192.23,406.51,22436.48,13398.73,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28333,30251,1136,50,1.0,0.0,1,1598.0,3,220,0.0,...,4870.82,3498.47,63.49,0.13,0.13,0.13,14.41,4873.09,4882.15,0
28337,30255,1520,59,1.0,0.0,2,162.0,2,860,3.0,...,10117.04,9609.18,8806.92,321.46,321.46,0.03,0.03,9536.99,9826.72,0
28339,30257,5298,56,1.0,0.0,1,1593.0,2,737,7.0,...,7633.96,7642.68,6007.43,1046.67,540.43,0.36,355.77,8066.92,7533.33,0
28341,30260,4567,60,1.0,0.0,1,0.0,2,1348,91.0,...,1425.74,1600.23,3726.50,0.46,0.46,0.46,0.46,1425.25,1439.10,0
